*Analytical Information Systems*

# Tutorial 2 - Data Integration

Matthias Griebel<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

SS 2019

<h1>Agenda<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-Recap-of-Data-Integration" data-toc-modified-id="1-Recap-of-Data-Integration-1">1 Recap of Data Integration</a></span></li><li><span><a href="#2-Data-Extraction" data-toc-modified-id="2-Data-Extraction-2">2 Data Extraction</a></span></li><li><span><a href="#3-Data-Transformation" data-toc-modified-id="3-Data-Transformation-3">3 Data Transformation</a></span><ul class="toc-item"><li><span><a href="#3.1-Data-Cleansing" data-toc-modified-id="3.1-Data-Cleansing-3.1">3.1 Data Cleansing</a></span></li><li><span><a href="#3.2-Data-Harmonization" data-toc-modified-id="3.2-Data-Harmonization-3.2">3.2 Data Harmonization</a></span></li><li><span><a href="#3.3-Data-Combination" data-toc-modified-id="3.3-Data-Combination-3.3">3.3 Data Combination</a></span></li><li><span><a href="#3.4-Enrich-the-data" data-toc-modified-id="3.4-Enrich-the-data-3.4">3.4 Enrich the data</a></span></li></ul></li><li><span><a href="#4-Data-Loading" data-toc-modified-id="4-Data-Loading-4">4 Data Loading</a></span></li><li><span><a href="#5-Exam-Questions" data-toc-modified-id="5-Exam-Questions-5">5 Exam Questions</a></span><ul class="toc-item"><li><span><a href="#5.1-Exam-AIS-SS-2018,-Question-1" data-toc-modified-id="5.1-Exam-AIS-SS-2018,-Question-1-5.1">5.1 Exam AIS SS 2018, Question 1</a></span></li></ul></li></ul></div>

## 1 Recap of Data Integration

#### Definition [Gartner IT Glossary](https://www.gartner.com/it-glossary/data-integration-tools/)

*'[...] __data integration__ comprises the practices, architectural techniques and tools for achieving the __consistent access and delivery__ of data across the spectrum of data subject areas and data structure types in the enterprise to meet the data consumption requirements of all applications and business processes.'*

#### Identify relevant databases

- First step of data integration
- External and internal data sources comprise 
    - relational and non-relational databases, 
    - XML, JSON
    - flat files (e.g., .csv)
    - and many more<br>

- Two types of heterogeneity will inevitably emerge: Schema and data-level<br>

__How do get the data ready for analysis?__

<img src="images/02/BIStack_ds.png" style="width:80%">



#### Data Warehousing: ETL

*from [Wikipedia](https://en.wikipedia.org/wiki/Extract,_transform,_load)*:

"[...] extract, transform, load (ETL) is the general procedure of copying data from one or more sources into a destination system which represents the data differently from the source(s)."
- __Data extraction__ involves extracting data from homogeneous or heterogeneous sources
- __Data transformation__ processes data by data cleansing and transforming them into a proper storage format/structure for the purposes of querying and analysis
- __Data loading__ describes the insertion of data into the final target database such as an operational data store, a data mart, or a data warehouse.

<img src="images/02/BIStack_etl.png" style="width:80%">

#### Today's Focus

We will learn how to...
- read data from different sources
- cleanse and transform the data sets
- combine the data sets
- enrich the data sets
- load the data into a database management system
- perform queries on a database 

## 2 Data Extraction

<img src="https://readr.tidyverse.org/logo.png" style="width:15%; float:right">

- The **readr** package provides a fast and friendly way <br> for reading rectangular data 
    - File formats: .csv, .tsv, and .fwf
    - Part of the core tidyverse
```R
# Example
read_csv("file.csv")
```


- **Packages for other file formats**
    - haven - SPSS, Stata, and SAS files 
    - readxl - excel files (.xls and .xlsx) 
    - DBI - databases
    - jsonlite - json
    - xml2 - XML
    - httr - Web APIs
    - rvest - HTML (Web Scraping)

#### Data Import Cheat Sheet
<img src="https://www.rstudio.com/wp-content/uploads/2018/08/data-import-600x464.png" style="width:50%; float:right">

- How to read in flat files 
- Work with the results as tibbles
- Reshape messy data

[Download here](https://github.com/rstudio/cheatsheets/raw/master/data-import.pdf)



#### *Up to you - Read the data*

You are provided with a set of operational data from a retail company. 

- Transaction data (Comma Delimited Files): 
    - *'transactions_eng.csv'*
    - *'transactions_ger.csv'*
   
- Customer data (Semi-colon Delimited Files)
    - *'customers.csv*
    - *'customers_usa.csv*

- Product data (Excel Files)
    - *'products_convenience.xlsx'*
    - *'products.xlsx*
    
Load the provided files. All files are stored in the folder *'data/T02/'*
- you can use `setwd()` to change your working directory to 'data/T02'

- Load the required packages

In [ ]:
library(tidyverse) # includes the readr package
library(readxl) # excel files

- Change the working directory

In [ ]:
setwd('data/T02')

- Read the data using the appropriate functions

- Take a look at the data 
```R
head()
sample_n()
```
- Check that all data is read in correctly
```R
nrow(), ncol(), colnames()
```
- Understand the rows and columns (observations and variables)
```R
glimpse()
summary()
```

## 3 Data Transformation

Data transformation converts the data into a proper storage format/structure for the purposes of querying and analysis
- Data cleansing 
    - Correct syntactical or semantical defects
- Data harmonization
    - Solving schema heterogeneity 
    - Solving data-level heterogeneity
- Data combination
    - Combine the harmonized data sets
- Data enrichment
    - Calculate frequently required (business) key figures as separate attributes

### 3.1 Data Cleansing 

__Correction of syntactical or semantical defects__

Depending on the degree of automation in the defect detection and defect correction we distinguish between three classes of defects.<br><br>

| <br>               | Automated Correction         | Manual Correction            |
|--------------------|------------------------------|------------------------------|
| Automated Detection| 1<sup>st</sup> class defects | 2<sup>nd</sup> class defects |
| Manual Detection   | -                            | 3<sup>rd</sup> class defects |

#### 1<sup>st</sup> class deficiencies

Syntactic and semantic deficiencies that are known in advance or can be anticipated.
- Syntactic: format, special characters, …
- Semantic: missing values in operating systems due to planned maintenance, …

→ Deficiencies can be automatically resolved by implementing transformation rules

#### *Up to you - 1<sup>st</sup> class deficiencies*

1. Find the syntactic 1st class deficiencies in the products convenience data sets
    - `products_convenience`: look at price and costs
    - `customers_usa`: look at the names

#### *Up to you - 1<sup>st</sup> class deficiencies*  

2. Implement transformation rules to resolve the deficiencies
    - Transformation rules can be implemented as pipes
    - You will have to use mutate() in combination with *str_replace()* and/or *str_split()*

#### 2<sup>nd</sup> class deficiencies

Deficiencies that can be automatically identified but have to be removed manually

- Syntactic: prior unknown syntactic error are identified for the first time and transferred into transformation rules
- Semantic: deficiencies can be identified by automatic plausibility checks or pattern recognition algorithms

→ Deficiencies are usually due to errors in the data source, corrective action must be taken at the operational source.

#### *Up to you - 2<sup>nd</sup> class deficiencies*

1. Perform plausibility checks (min, mean, max, …) to identify deficiencies in the product data 


#### *Up to you - 2<sup>nd</sup> class deficiencies*

2. Implement transformation rules to resolve the deficiencies. <br> If you identify errors or missing values you can either:
    - Keep the errors / missing values
    - Remove the observations
    - Impute the values

- remove

- impute

#### 3<sup>rd</sup> class deficiencies

Semantic deficiencies that can only be manually identified and resolved.
- Errors in the data set that can only be identified by domain experts

→ Prompt correction of the deficiencies can be supported by software tools (Workflow-Management-Systems)

#### *Up to you - 3<sup>rd</sup> class deficiencies*

1. Find the semantic 3<sup>rd</sup> class deficiencies in the customer data
    - `costomer`: Take a closer look at the countries

#### *Up to you - 3<sup>rd</sup> class deficiencies*

2. Resolve the deficiencies

### 3.2 Data Harmonization

#### Solving Schema Heterogeneity

Schemas are created by different people whose states and styles are different
- Same concept, but different names for tables and attributes
    - rating vs classification
- Multiple attributes in first schema relate to a single attribute in the other
    - basePrice and taxRate relate to price
- Tabular organization of schemas can be quite different
    - One table in DB1 vs three tables in DB2
- Coverage and level of details can also differ significantly
    - Daily sales data vs. monthly sales data
    - High-level information (movie name, playtime) vs. detailed meta data (director, genre, rating)


#### *Up to you - Data Harmonization*

1. Find and harmonize schema heterogeneity in transaction data sets
    - Look at the attribute names
    - Adjust the transactions_ger to the schema of transactions_eng

#### Solving Data-level  Heterogeneity
Data coming from different sources rarely joins perfectly

Typical reasons include
- Differently coded data
    - identical attribute names and identical meanings but different domains or value ranges (e.g., gender coded as “m / f” or “male / female”)
- Synonyms
    - Attributes that have different names but the same meaning and domain (e.g., “client” or “customer”)
- Homonyms: 
    - Attributes have the same attribute names, but have different meanings (e.g., “partner” = “customer” or “partner” = “supplier”)

#### *Up to you - Data Harmonization*

2. Find and harmonize data-level heterogeneity in the customer data sets
    - Take a closer look to the variables names as well as the variable values
    - Adjust the `customers_usa` to the schema of `customers`

### 3.3 Data Combination

Combine the harmonized data sets into an appropriate format/structure for further analysis.

#### *Up to you - Combine the data* 

1. Combine the harmonised data sets
    - Create three new data sets: `customers`, `transactions`, `products`
    - Use *bind_rows()* for binding multiple data frames by row

#### *Up to you - Combine the data* 

2. Join the three data sets into one data final data frame

### 3.4 Enrich the data

- Business key figures are calculated and integrated into the data basis as separate attributes
- Example: Calculation of weekly contribution margins at product level and annual contribution margins at store level
- Advantages
    - Calculable response time behavior for later queries based on the advance calculation
    - Guaranteed consistency of the calculated values, since they are only formed once across all applications
    - Establishment of coordinated business management instruments

#### *Up to you - Enrich the data* 

2. Create two new variables:
    - revenue per transaction
    - profit per transaction

## 4 Data Loading

__Data loading__ describes the insertion of data into the final target database such as an operational data store, a data mart, or a data warehouse (e.g., AWS Redshift or Apache Hive)

- A database is an organized collection of data
    - typically cleaned and structured data
- The data is stored 
- Database management system (DBMS)
    - allows to store, modify, and extract information from a database MySQL, ORACLE, MS SQL, DB2, MS ACCESS, Informix
- RDBMS stands for Relational Database Management System
    - data is stored in database objects called tables
    - tables can be joined through keys and indexes
- SQL is used to work with DBMS

#### What is SQL?

- A standard language for accessing and manipulating data in databases
    - SQL stands for Structured Query Language
    - allows retrieving and manipulating data as well as administrative operations

- A typical SQL command
```SQL
	SELECT <column names separated by comma>
    FROM <database table>
	WHERE <condition>
    GROUP BY <column name>
    ORDER BY <column name>
```

#### Using differnte query languages in R

- The *DBI* (Database Interface) package allows communication between R and relational database management systems
    - Using a DBI-compliant interface, the *RSQLite* embeds the SQLite database engine in R
    - SQLite is a public-domain, single-user, very light-weight database engine that implements a decent subset of the SQL 92 standard,
- The packages *dbplyr* (part of the *tidyverse*) is designed to work with database tables as if they were local data frames


<img src="https://github.com/matjesg/AIS_2019/raw/master/notebooks/images/01/aris.png" style="width:30%">

We’ll first create an in-memory SQLite database

In [ ]:
con <- DBI::dbConnect(RSQLite::SQLite(), ":memory:")

and copy over our dataset

In [ ]:
copy_to(con, data_enriched)

Now you can retrieve a table using `tbl()` . Printing it just retrieves the first few rows:

In [ ]:
db <- tbl(con, "data_enriched")
db

- (Lazily) generate query

In [ ]:
db %>%
    filter(payment == 'cash') %>%
    summarise(MeanAmount = mean(amount, na.rm = TRUE)) -> summary

- See query

In [ ]:
summary %>% show_query()

- Execute query and retrieve results

In [ ]:
summary %>% collect()

#### *Up to you - Analyze the data* 

How much profit did the company realize in 2017?

## 5 Exam Questions

### 5.1 Exam AIS SS 2018, Question 1

__Data Engineering & Integration (10 points)__


(a) __Getting orders in order__: You are working for a major online retailer who is interested in optimizing internal logistics processes. A key problem in this context is the handling of orders with a single line item vs. orders with multiple line items.

The cornerstone of your analysis is an orders table with the following structure:

| productID | quantity | orderID |
|-----------|----------|---------|
| ...       | ...      | ...     |

i. (1 points) Explain (verbally or in pseudo code) how you would identify the number of orders with a single line item from this data base.

i. (2 points) The frontend reporting tool used by the logistics department cannot handle data sets with more than 1 million rows. Yet your order table has many more rows. Recognizing that individual product IDs are not crucial for the logistics process analysis (handling times are determined by the number of products in an order) you are approached to provide a \textit{compact representation which retains the structure (number of line items) of the order invoices}. Explain how this can be achieved by means of clever aggregation.

(b) (3 Points) __Here comes the sun__: You are working for a local crime investigation unit and your current assignment involves a series of burglary cases. The head of the investigation wants to know which share of the of the break-ins took place during night-time (i.e., between sunset and sunrise). You are provided with a table of the time and date for the burglary events as well as a table of sunrise and sunset times for all dates in the time period under consideration. Provide an analytic pipeline (verbally or in pseudo code) to answer this question.

| date | time | crimeID |
|------|------|---------|
| ...  | ...  | ...     |

| date | sunriseTime | sunsetTime |
|------|-------------|------------|
| ...  | ...         | ...        |

(c) (2 points) __Difficulties__ of data integration arise from different perspectives. Discuss the systems-level as well as social reasons which render data integration a hard task.

(d) (2 points) __Scalability__ of naively executed string matching operations is problematic for large data sets. Explain why. What is a typical workaround?